# Creating Database and Table

In [1]:
"""Illustrate a many-to-many relationship between an
"Order" and a collection of "Item" objects, associating a purchase price
with each via an association object called "OrderItem"

The association object pattern is a form of many-to-many which
associates additional data with each association between parent/child.

The example illustrates an "order", referencing a collection
of "items", with a particular price paid associated with each "item".

"""

import sqlalchemy as db
import pandas as pd

# from datetime import datetime

import datetime

from sqlalchemy import and_

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, String, Integer, Date, Table, ForeignKey, Boolean
from sqlalchemy.orm import relationship, backref
from sqlalchemy import DateTime
from sqlalchemy import Column
from sqlalchemy import create_engine
from sqlalchemy import DateTime
from sqlalchemy import Float
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String

import daiquiri
import daiquiri.formatter

import logging
# logging.basicConfig(level=logging.DEBUG)

FMT_SIMPLE = "%(asctime)-15s %(name)-5s %(levelname)-8s "
"PID: %(process)d THREAD: %(threadName)s ProcessName: %(processName)s "
"FILE: %(filename)s:%(lineno)s FUNCTION: %(funcName)s %(message)s"

def setup_logging(level=None, outputs=None, fmt=FMT_SIMPLE):
    """Configure logging."""
    if not level:
        level = logging.DEBUG
    if not outputs:
        # outputs = (daiquiri.output.STDOUT,)
        outputs = (
            daiquiri.output.Stream(
                formatter=daiquiri.formatter.ColorFormatter(fmt=fmt)
            ),
        )
    daiquiri.setup(level=level, outputs=outputs)


# datefmt='%Y-%m-%d %H:%M:%S'



SQLALCHEMY_POOL_LOGGER = logging.getLogger("sqlalchemy.pool")
SQLALCHEMY_ENGINE_LOGGER = logging.getLogger("sqlalchemy.engine")
SQLALCHEMY_ORM_LOGGER = logging.getLogger("sqlalchemy.orm")
SQLALCHEMY_DIALECTS_LOGGER = logging.getLogger("sqlalchemy.dialects")
SQLALCHEMY_POOL_LOGGER.setLevel(logging.DEBUG)
SQLALCHEMY_ENGINE_LOGGER.setLevel(logging.DEBUG)
SQLALCHEMY_ORM_LOGGER.setLevel(logging.DEBUG)
SQLALCHEMY_DIALECTS_LOGGER.setLevel(logging.DEBUG)


# logger = logging.getLogger(__name__)

setup_logging()

logger = daiquiri.getLogger(__name__)
logger.info("It works with a custom format!")

def debug_dump_exclude(obj, exclude=["__builtins__", "__doc__"]):
    for attr in dir(obj):
        if hasattr(obj, attr):
            if attr not in exclude:
                print("obj.%s = %s" % (attr, getattr(obj, attr)))

2019-08-09 19:57:42,473 __main__ INFO     


In [2]:
# Creates: a SQLAlchemy Engine that will interact with our sqlite database,
engine = db.create_engine('sqlite:///learn.sqlite')

# Creates: a SQLAlchemy ORM session factory bound to this engine
Session = sessionmaker(bind=engine)

# Creates: a base class for our classes definition
Base = declarative_base()


connection = engine.connect()
metadata = db.MetaData()
# census = db.Table('census', metadata, autoload=True, autoload_with=engine)

2019-08-09 19:57:42,507 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:42,510 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:42,513 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:42,520 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:42,522 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:42,525 sqlalchemy.pool.impl.NullPool DEBUG    


In [3]:
# emp = db.Table('emp', metadata,
#               db.Column('Id', db.Integer()),
#               db.Column('name', db.String(255), nullable=False),
#               db.Column('salary', db.Float(), default=100.0),
#               db.Column('active', db.Boolean(), default=True)
#               )

# metadata.create_all(engine) #Creates the table

In [4]:
class Order(Base):
    __tablename__ = "order"

    order_id = Column(Integer, primary_key=True)
    customer_name = Column(String(30), nullable=False)
    order_date = Column(String(255), nullable=False)
    order_items = relationship(
        "OrderItem", cascade="all, delete-orphan", backref="order"
    )

    def __init__(self, customer_name):
        self.customer_name = customer_name
        self.order_date = str(datetime.datetime.utcnow())


class Item(Base):
    __tablename__ = "item"
    item_id = Column(Integer, primary_key=True)
    description = Column(String(30), nullable=False)
    price = Column(Float, nullable=False)

    def __init__(self, description, price):
        self.description = description
        self.price = price

    def __repr__(self):
        return "Item(%r, %r)" % (self.description, self.price)


class OrderItem(Base):
    __tablename__ = "orderitem"
    order_id = Column(Integer, ForeignKey("order.order_id"), primary_key=True)
    item_id = Column(Integer, ForeignKey("item.item_id"), primary_key=True)
    price = Column(Float, nullable=False)

    def __init__(self, item, price=None):
        self.item = item
        self.price = price or item.price

    item = relationship(Item, lazy="joined")
    
    


2019-08-09 19:57:42,580 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,581 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,591 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,593 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,596 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,601 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,606 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,608 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,612 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,621 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,629 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,639 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,649 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,657 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,661 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:42,670 sqlalchemy.orm.mapper.Mapper IN

In [5]:
# # Print the column names
# print(emp.columns.keys())

In [6]:
# # Print full table metadata
# print(repr(metadata.tables['emp']))

# Inserting Data

In [7]:
# #Inserting record one by one
# query = db.insert(emp).values(Id=1, name='naveen', salary=60000.00, active=True) 
# ResultProxy = connection.execute(query)



In [8]:
# #Inserting many records at ones
# query = db.insert(emp) 
# values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
#                {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
# ResultProxy = connection.execute(query,values_list)

In [9]:
# results = connection.execute(db.select([emp])).fetchall()
# df = pd.DataFrame(results)
# df.columns = results[0].keys()
# df.head(4)

In [10]:
# Print the column names
# print(census.columns.keys())

In [11]:
# Print full table metadata
# print(repr(metadata.tables['census']))

# Querying

In [12]:
#Equivalent to 'SELECT * FROM census'
# query = db.select([census])

In [13]:
# ResultProxy = connection.execute(query)

In [14]:
# ResultSet = ResultProxy.fetchall()

In [15]:
# ResultSet[:3]

# Class: Movie

In [16]:
# # Source: https://auth0.com/blog/sqlalchemy-orm-tutorial-for-python-developers/

# # As many movies can have many actors and vice-versa, 
# # we will need to create a Many To Many relationship between these two classes.

# # we created a movies_actors_association table that connects rows of actors and rows of movies
# movies_actors_association = Table(
#     'movies_actors', Base.metadata,
#     Column('movie_id', Integer, ForeignKey('movies.id')),
#     Column('actor_id', Integer, ForeignKey('actors.id'))
# )


# class Movie(Base):
#     __tablename__ = 'movies'

#     id = Column(Integer, primary_key=True)
#     title = Column(String)
#     release_date = Column(Date)
#     # and we added the actors property to Movie and configured the 
#     # movies_actors_association as the intermediary table.
#     actors = relationship("Actor", secondary=movies_actors_association)

#     def __init__(self, title, release_date):
#         self.title = title
#         self.release_date = release_date

# Class: Actor

In [17]:
# class Actor(Base):
#     __tablename__ = 'actors'

#     id = Column(Integer, primary_key=True)
#     name = Column(String)
#     birthday = Column(Date)

#     def __init__(self, name, birthday):
#         self.name = name
#         self.birthday = birthday

# Class: Stuntman

In [18]:
# # In this class, we have defined that the actor property references an 
# # instance of Actor and that this actor will get a property called stuntman that is not a list (uselist=False). 
# # That is, whenever we load an instance of Stuntman, 
# # SQLAlchemy will also load and populate the Actor associated with this stuntman.

# class Stuntman(Base):
#     __tablename__ = 'stuntmen'

#     id = Column(Integer, primary_key=True)
#     name = Column(String)
#     active = Column(Boolean)
#     actor_id = Column(Integer, ForeignKey('actors.id'))
#     actor = relationship("Actor", backref=backref("stuntman", uselist=False))

#     def __init__(self, name, active, actor):
#         self.name = name
#         self.active = active
#         self.actor = actor

# Class: ContactDetails

In [19]:
# class ContactDetails(Base):
#     __tablename__ = 'contact_details'

#     id = Column(Integer, primary_key=True)
#     phone_number = Column(String)
#     address = Column(String)
#     actor_id = Column(Integer, ForeignKey('actors.id'))
#     actor = relationship("Actor", backref="contact_details")

#     def __init__(self, phone_number, address, actor):
#         self.phone_number = phone_number
#         self.address = address
#         self.actor = actor

# Persisting Data with SQLAlchemy ORM

In [20]:
# from datetime import date

# # 2 - generate database schema
Base.metadata.create_all(engine)

# # 3 - create a new session
session = Session()


2019-08-09 19:57:43,242 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:43,245 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:43,247 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,250 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,256 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,257 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,261 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,267 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,307 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,309 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,311 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,313 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,315 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,317 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,319 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,321 sqlalchemy.engin

In [21]:
# # 4 - create movies
# bourne_identity = Movie("The Bourne Identity", date(2002, 10, 11))
# furious_7 = Movie("Furious 7", date(2015, 4, 2))
# pain_and_gain = Movie("Pain & Gain", date(2013, 8, 23))


In [22]:
# # 5 - creates actors
# matt_damon = Actor("Matt Damon", date(1970, 10, 8))
# dwayne_johnson = Actor("Dwayne Johnson", date(1972, 5, 2))
# mark_wahlberg = Actor("Mark Wahlberg", date(1971, 6, 5))


In [23]:
# # 6 - add actors to movies
# bourne_identity.actors = [matt_damon]
# furious_7.actors = [dwayne_johnson]
# pain_and_gain.actors = [dwayne_johnson, mark_wahlberg]


In [24]:
# # 7 - add contact details to actors
# matt_contact = ContactDetails("415 555 2671", "Burbank, CA", matt_damon)
# dwayne_contact = ContactDetails("423 555 5623", "Glendale, CA", dwayne_johnson)
# dwayne_contact_2 = ContactDetails("421 444 2323", "West Hollywood, CA", dwayne_johnson)
# mark_contact = ContactDetails("421 333 9428", "Glendale, CA", mark_wahlberg)


In [25]:
# # 8 - create stuntmen
# matt_stuntman = Stuntman("John Doe", True, matt_damon)
# dwayne_stuntman = Stuntman("John Roe", True, dwayne_johnson)
# mark_stuntman = Stuntman("Richard Roe", True, mark_wahlberg)


In [26]:
# # 9 - persists data
# session.add(bourne_identity)
# session.add(furious_7)
# session.add(pain_and_gain)

# session.add(matt_contact)
# session.add(dwayne_contact)
# session.add(dwayne_contact_2)
# session.add(mark_contact)

# session.add(matt_stuntman)
# session.add(dwayne_stuntman)
# session.add(mark_stuntman)


In [27]:
# display data
# results = connection.execute(db.select([emp])).fetchall()
# df = pd.DataFrame(results)
# df.columns = results[0].keys()
# df.head(4)

In [28]:
# 10 - commit and close session
# session.commit()
# session.close()

In [29]:
debug_dump_exclude(Base)

obj.__class__ = <class 'sqlalchemy.ext.declarative.api.DeclarativeMeta'>
obj.__delattr__ = <slot wrapper '__delattr__' of 'object' objects>
obj.__dict__ = {'_decl_class_registry': <WeakValueDictionary at 0x1157ce5f8>, 'metadata': MetaData(bind=None), '__doc__': 'The most base type', '__init__': <function _declarative_constructor at 0x11565a7b8>, '__module__': 'sqlalchemy.ext.declarative.api', '__dict__': <attribute '__dict__' of 'Base' objects>, '__weakref__': <attribute '__weakref__' of 'Base' objects>}
obj.__dir__ = <method '__dir__' of 'object' objects>
obj.__eq__ = <slot wrapper '__eq__' of 'object' objects>
obj.__format__ = <method '__format__' of 'object' objects>
obj.__ge__ = <slot wrapper '__ge__' of 'object' objects>
obj.__getattribute__ = <slot wrapper '__getattribute__' of 'object' objects>
obj.__gt__ = <slot wrapper '__gt__' of 'object' objects>
obj.__hash__ = <slot wrapper '__hash__' of 'object' objects>
obj.__init__ = <function _declarative_constructor at 0x11565a7b8>
obj

In [30]:
# metadata.create_all(engine) #Creates the table

In [31]:
# create catalog
tshirt, mug, hat, crowbar = (
    Item("SA T-Shirt", 10.99),
    Item("SA Mug", 6.50),
    Item("SA Hat", 8.99),
    Item("MySQL Crowbar", 16.99),
)
session.add_all([tshirt, mug, hat, crowbar])
session.commit()

# create an order
order = Order("john smith")

2019-08-09 19:57:43,580 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:43,584 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:43,591 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,594 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,597 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,598 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,602 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,605 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,607 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,609 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,611 sqlalchemy.orm.mapper.Mapper INFO     
2019-08-09 19:57:43,614 sqlalchemy.orm.relationships.RelationshipProperty INFO     
2019-08-09 19:57:43,616 sqlalchemy.orm.relationships.Re

In [34]:
debug_dump_exclude(order)

obj.__class__ = <class '__main__.Order'>
obj.__delattr__ = <method-wrapper '__delattr__' of Order object at 0x115832fd0>
obj.__dict__ = {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x115832b70>, 'customer_name': 'john smith', 'order_id': 1, 'order_date': '2019-08-09 23:57:43.724492', 'order_items': [<__main__.OrderItem object at 0x1158ce6d8>, <__main__.OrderItem object at 0x1158ce898>, <__main__.OrderItem object at 0x1158ce9b0>]}
obj.__dir__ = <built-in method __dir__ of Order object at 0x115832fd0>
obj.__eq__ = <method-wrapper '__eq__' of Order object at 0x115832fd0>
obj.__format__ = <built-in method __format__ of Order object at 0x115832fd0>
obj.__ge__ = <method-wrapper '__ge__' of Order object at 0x115832fd0>
obj.__getattribute__ = <method-wrapper '__getattribute__' of Order object at 0x115832fd0>
obj.__gt__ = <method-wrapper '__gt__' of Order object at 0x115832fd0>
obj.__hash__ = <method-wrapper '__hash__' of Order object at 0x115832fd0>
obj.__init__ = <boun

In [32]:
# add three OrderItem associations to the Order and save
order.order_items.append(OrderItem(mug))
order.order_items.append(OrderItem(crowbar, 10.99))
order.order_items.append(OrderItem(hat))
session.add(order)
session.commit()

2019-08-09 19:57:43,732 sqlalchemy.orm.path_registry DEBUG    
2019-08-09 19:57:43,736 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:43,738 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:43,741 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,744 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,745 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,747 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,749 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,751 sqlalchemy.orm.path_registry DEBUG    
2019-08-09 19:57:43,753 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,754 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,755 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,758 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,762 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,763 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,766 sqlalchemy.orm.pat

In [33]:
# query the order, print items
order = session.query(Order).filter_by(customer_name="john smith").one()
print(
    [
        (order_item.item.description, order_item.price)
        for order_item in order.order_items
    ]
)

# print customers who bought 'MySQL Crowbar' on sale
q = session.query(Order).join("order_items", "item")
q = q.filter(
    and_(Item.description == "MySQL Crowbar", Item.price > OrderItem.price)
)

print([order.customer_name for order in q])

2019-08-09 19:57:43,805 sqlalchemy.orm.path_registry DEBUG    
2019-08-09 19:57:43,809 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:43,814 sqlalchemy.pool.impl.NullPool DEBUG    
2019-08-09 19:57:43,816 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,819 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,821 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,825 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,827 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,829 sqlalchemy.orm.path_registry DEBUG    
2019-08-09 19:57:43,832 sqlalchemy.orm.path_registry DEBUG    
2019-08-09 19:57:43,852 sqlalchemy.orm.path_registry DEBUG    
2019-08-09 19:57:43,860 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,863 sqlalchemy.engine.base.Engine INFO     
2019-08-09 19:57:43,867 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,877 sqlalchemy.engine.base.Engine DEBUG    
2019-08-09 19:57:43,880 sqlalchemy.engine.ba

[('SA Mug', 6.5), ('SA Hat', 8.99), ('MySQL Crowbar', 10.99)]
['john smith']
